In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [17]:
data = []

url = 'https://suumo.jp/jj/common/ichiran/JJ901FC004/?initFlg=1&seniFlg=1&pc=30&ar=030&ra=030013&rnTmp=0005&ct=120.0&kb=0&xb=0&et=5&cn=10&newflg=0&km=1&rn=0005&bs=040'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

body = soup.find("body")
pages = body.find_all("div", {'class':'pagination pagination_set-nav'})
pages_text = str(pages)
pages_split = pages_text.split('</a></li>\n</ol>')
num_pages = int(pages_split[0].split('>')[-1])

for i in range(1,num_pages):
    url = 'https://suumo.jp/jj/common/ichiran/JJ901FC004/?initFlg=1&seniFlg=1&pc=30&ar=030&ra=030013&rnTmp=0005&ct=120.0&kb=0&xb=0&et=5&cn=10&newflg=0&km=1&rn=0005&bs=040&pn={}'.format(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    bukkens = soup.find_all('li',attrs={'class':'cassette js-bukkenCassette'})

    for bukken in bukkens:
        bukken_info = bukken.select(".infodatabox-box-txt")
        # 沿線
        railline = bukken_info[1].text.split('/')[0]
    #     最寄駅
        station = bukken_info[1].text.split('/')[1]
        # 駅までの所要時間
        how_long = bukken_info[2].text
        how_long = int(re.sub(r"\D", "", how_long) )
        # 家賃
        rentprice = bukken.find('dd',attrs={'class':'infodatabox-details-txt'}).text
        rentprice = float(rentprice.replace('万円',''))
        # 占有面積
        price = bukken.find_all('dd',attrs={'class':'infodatabox-details-txt'})
        area = price[3].text
        area = float(re.sub(r"[^\d.]", "", area) )
        # 間取り
        room_type = price[4].text.replace('\r\n','').replace('\t','')

        datum = {}
        datum['沿線']=railline
        datum['最寄駅']=station
        datum['駅までの所要時間（分）']=how_long
        datum['家賃（万円）']=rentprice
        datum['占有面積']=area
        datum['間取り']=room_type
        data.append(datum)

In [18]:
import pandas as pd
df = pd.DataFrame(data)
df

,沿線,最寄駅,駅までの所要時間（分）,家賃（万円）,占有面積,間取り
0,ＪＲ山手線,新大久保,5,6.90,9.012,ワンルーム
1,ＪＲ山手線,田端,5,6.95,12.820,ワンルーム
2,ＪＲ山手線,大崎,5,7.60,16.220,1K
3,ＪＲ山手線,大崎,5,7.80,16.220,1K
4,ＪＲ山手線,鶯谷,2,7.80,17.922,ワンルーム
...,...,...,...,...,...,...
20125,ＪＲ山手線,浜松町,5,80.00,80.392,3LDK
20126,ＪＲ山手線,浜松町,5,80.00,80.392,3LDK
20127,ＪＲ山手線,浜松町,5,80.00,80.392,3LDK
20128,ＪＲ山手線,浜松町,5,80.00,80.392,3LDK


In [19]:
df.columns

Index(['沿線', '最寄駅', '駅までの所要時間（分）', '家賃（万円）', '占有面積', '間取り'], dtype='object')

In [20]:
df.to_csv('suumo_yamanote_toho5min_chikunenn10years.csv',index=None)